In [ ]:
# import libraries to request data from URLs and read them
import requests
from bs4 import BeautifulSoup
import os
import pandas as pd

#URL of directory of weather data
weather_data_url = "https://www.ncei.noaa.gov/data/global-summary-of-the-day/access/"
#get HTML text from ULR
url_text = requests.get(weather_data_url)
#use Beautiful Soup to parse data in HTML text
weather_soup = BeautifulSoup(url_text.text,'html.parser')

#create list of all year folders containing .csv files 
year_folders = [a['href'] for a in weather_soup.find_all('a', href=True) if a['href'].endswith('/') and not a['href'].startswith('/')]

#create Data Frame to fill from csv files
used_columns = ['DATE','LATITUDE','LONGITUDE','ELEVATION','NAME','FRSHTT']
#weather_df=pd.DataFrame([['init',1.1,1.1,1.1,'init',1]],columns=used_columns)

for year in year_folders:
    #year="1929/"
    if os.path.exists(f'{year[:len(year)-1]}_data.csv'):
        continue
    year_df = pd.DataFrame([['init',1.1,1.1,1.1,'init',1]],columns=used_columns)
    #request and create beautiful soup for each year folder
    year_text = requests.get(weather_data_url+year)
    year_soup = BeautifulSoup(year_text.text,'html.parser')
    #create list of all csv files in each folder
    csv_files = [a['href'] for a in year_soup.find_all('a', href=True) if a['href'].endswith('.csv')]
    total_files = len(csv_files)
    current_file = 0
    for file in csv_files:
        #print(weather_data_url+year+file)
        current_file = current_file + 1
        print(f"\rLoading file: {current_file}/{total_files} in year {year}          ", end = "", flush = True)
        csv_df = pd.read_csv(weather_data_url+year+file,usecols=used_columns)
        #print(csv_df.info())
        year_df=pd.concat([year_df,csv_df], ignore_index = True)
    year_df=year_df.drop(0)
    year_df.to_csv(f'{year[:len(year)-1]}_data.csv',index=False)

print('Complete')

#weather_df=weather_df.drop(0)
#weather_df.info()
#weather_df.to_csv('Weather_Data.csv',index=False)
